## **SISTEMA RECOMENDADOR DE NOTICIAS PARA CLIENTES CORPORATIVOS DE UN BANCO**


**Profesor**: Juan Fernando Perez Bernal

**Integrantes**: Miller Puentes, Joan Esteban Chacón, Grace González, Catalina Cárdenasace González, Catalina Cárdenas


En este documento se presenta el Modelamiento de los datos para el desarrollo de un sistema de recomendación de noticias para los clientes de un banco basado en el sector económico al que pertenece.

### ***Tabla de contenido***

**0.   Instalación y llamada de librerías**

  0.1 Instalación de pyLDAvis

  0.2 Instalación de librerías


**4. Recomendadores**

  4.1 Recomendador: LDA - Stemming

  4.2 Recomendador: LDA - Lematización

**5. Resultados**


## ***0. Instalación y llamada de librerías***

### ***0.1 Instalación de pyLDAvis***

In [1]:
import pandas as pd
import numpy as np
import pyLDAvis

print(pd.__version__)
print(np.__version__)
print(pyLDAvis.__version__)

2.0.0
1.24.3
3.4.0


### ***0.2 Instalación de librerías***

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Importación de librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import unidecode
import spacy
from spacy.lang.es.examples import sentences
from unidecode import unidecode
import zipfile
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import nltk
import pyLDAvis.gensim_models
from gensim.corpora import Dictionary
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.decomposition import LatentDirichletAllocation
from wordcloud import WordCloud
# from transformers import AutoModel, AutoTokenizer
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
from wordcloud import WordCloud
from pprint import pprint
from gensim.models import LdaModel

# Visualizamos los resultados
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

# Cargar el modelo de spacy para el español
nlp = spacy.load("es_core_news_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\relat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\relat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\relat\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\relat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## ***4. Recomendadores***

Para contruir el recomendador se usa un sistema de prioridad de acuerdo al sector al que pertenece el cliente, pero se hace una distinción dependiendo del sector al que pertenece el cliente como se muestra a continuación:

* ***Sector Industrial:***
       'manufactura de productos de tocador articulos de aseo',
       'combustibles lubricantes', 'comercio de repuestos',
       'edificaciones', 'gas', 'distribuidores productos consumo masivo',
       'automotores', 'grasas aceites', 'obras de infraestructura',
       'comercio de electrodomesticos', 'ferreteria', 'papel carton',
       'extraccion de carbon', 'plastico', 'maquinaria',
       'imprentas edicion', 'otros servicios financieros',
       'transporte terrestre', 'bancos', 'supermercados',
       'quimicos basicos otros quimicos', 'madera muebles',
       'servicios empresas', 'carnes frias',
       'productos metalicos', 'transporte maritimo', 'agroquimicos',
       'comisionistas de bolsa', 'cacao', 'servicios personas',
       'establecimientos educativos', 'comercio de variedades vestuario',
       'calzado marroquineria', 'confecciones',
       'acueducto alcantarillado', 'medios', 'fondos de pensiones',
       'transporte aereo', 'fiduciarias', 'molineria derivados',
       'alimentos concentrados', 'seguros', 'valor agregado',
       'extraccion de petroleo', 'textiles', 'electricidad', 'conservas',
       'ganaderia porcicultura', 'cajas de compensacion',
       'industria basica de hierro acero', 'productos lacteos',
       'pesca atun', 'aseo', 'otras actividades mineras',
       'companias de financiamiento comercial', 'flores',
       'comercio de quimicos agroquimicos', 'banano',
       'productos ceramicos', 'vidrio', 'telefonia fija',
       'bebidas no alcoholicas', 'equipo de transporte',
       'repuestos caucho', 'hoteles', 'cemento', 'bebidas alcoholicas',
       'palma africana', 'otros cultivos', 'otros manufactura',
       'resinas plasticas', 'papeles suaves', 'avicultura', 'arroz',
       'confiteria', 'azucar', 'tabaco', 'comercio de cafe',
       'pinturas barnices'

En el sector industrial la prioridad de las noticias se encuentran por:

1.   `Macroeconómicas`
2.   `Politica`
3.   `Sostenibilidad`
4.   `Innovación`
5.   `Regulaciones`
6.   `Alianzas`
7.   `Reputación`
8.   `Cultura`
9.   `Salud`
10.  `Deporte`

* Sector Salud:
       'ips','comercio de farmaceuticos', 'eps salud prepagada salud',
       'manufactura de productos farmaceuticos'

En el sector de la salud la prioridad y los pesos están función de:

1.   `Salud`
2.   `Sostenibilidad`
3.   `Innovación`
4.   `Regulaciones`
5.   `Deporte`
6.   `Cultura`
7.   `Reputación`
8.   `Macroeconómicas`
9.   `Politica`
10.  `Alianzas`


Nota: La categoría 'Otra' se indica que la eliminemos de las sugerencias ya que no son relevantes para el negocio.

### ***4.1 Recomendador: LDA - Stemming***

In [4]:
df_bd_stemming_f2=pd.read_csv('../Data/DC_stemming_tema_asignado.csv')

Se crea la columna con el sector y se seleccionan las variables más relevantes para el recomendador:

In [5]:
condicion = lambda x: 'Salud' if x in ['ips','comercio de farmaceuticos', 'eps salud prepagada salud','manufactura de productos farmaceuticos'] else 'Industrial'
df_bd_stemming_f2['sector'] = df_bd_stemming_f2['subsec'].apply(condicion)

# Se seleccionana las columnas relevantes
df_bd_stemming_f2 = df_bd_stemming_f2[['nit','sector', 'subsec','news_id', 'news_title', 'news_text_content', 'Tema_texto']]
df_bd_stemming_f2.head(3)

,nit,sector,subsec,news_id,news_title,news_text_content,Tema_texto
0,800000946,Industrial,manufactura de productos de tocador articulos ...,news15986,Guia de Arte y Cultura: semana del 12 al 19 de...,"ARTE Agustina Galindez Quesada exhibe ""Catarsi...",Regulaciones
1,800011161,Industrial,automotores,news17955,"No solo habra desfiles el 20 de julio, estos s...","El Dia de la Independencia, este miercoles 20 ...",Regulaciones
2,800015615,Industrial,ferreteria,news96447,Black Friday | Comerciantes posadenos prometen...,Desde este jueves se realizara en Posadas la y...,Regulaciones


Se revisa cuantas noticias existe por sector:

In [6]:
df_bd_stemming_f2['sector'].value_counts()

sector
Industrial    21697
Salud          1303
Name: count, dtype: int64

Gran parte de la población pertenece a otros sectores diferentes al sector salud.

Ahora, se procede a crear las funciones del recomendador:

1. seleccionar_dos_filas_aleatorias: esta función seleccionará dos filas aleatorias de un grupo de noticias.
2. asignar_valor: Esta función asignará el valor que se le debe dar a la noticia de acuerdo a las definiciones del negocio y del sector del cliente.
3. recomendador: Esta función toma como insumo el nit de un cliente, el número de noticias que se le quiere mostrar y el dataframe que se le va a suministrar. La función filtra por su sector y le asigna la relevancia a la notica de acuerdo a la función `asignar_valor` que está en función del sector al que pertenece, para luego ordenarlas por temas y seleccionar 2 noticias de cada topico para presentarla al cliente, también permite visualizar el número de noticias que se desea ver (pero priorizando las de mayor relevancia para el cliente).

In [7]:
def seleccionar_dos_filas_aleatorias(group):
    return group.sample(n=min(2, len(group)))

def asignar_valor(row, sector):
    temas = {
        'Salud': [1, 9],
        'Sostenibilidad': [2, 3],
        'Innovación': [3, 4],
        'Regulaciones': [4, 5],
        'Deportes': [5, 10],
        'Cultura': [6, 8],
        'Reputación': [7, 7],
        'Macroeconómicas': [8, 1],
        'Política': [9, 2],
        'Alianzas': [10, 6]
    }

    tema = row['Tema_texto']

    if sector == 'Salud':
        return temas.get(tema, [11, 0])[0]
    else:
        return temas.get(tema, [11, 0])[1]

def recomendador(nit, num_noticias, dff):
    # Se copia el df ingresado
    dff1 = dff.copy()

    # Se retiran las noticias asignadas como 'Otra'.
    dff1 = dff1[dff1['Tema_texto'] != 'Otra']

    # Se identifica el sector del Nit
    s = dff1[dff1['nit'] == nit]['sector'].iloc[0]

    # Se le aplica el valor asignado con la función asignar_valor
    dff1['valor_asignado'] = dff1.apply(lambda row: asignar_valor(row, s), axis=1)

    # Ordena el DataFrame por la columna 'valor_asignado'
    dff1 = dff1.sort_values(by='valor_asignado')

    # Agrupa el DataFrame por 'valor_asignado' y aplica la función para seleccionar aleatoriamente dos noticias
    resultados = dff1.groupby('valor_asignado', group_keys=False).apply(seleccionar_dos_filas_aleatorias)

    resultados = resultados[['news_id', 'news_title', 'Tema_texto', 'news_text_content']]

    resultados.reset_index(drop=True,inplace=True)

    return resultados.head(num_noticias)

In [8]:
# Ejemplo cliente sector salud
p = recomendador(860003216, 10, df_bd_stemming_f2)
p

,news_id,news_title,Tema_texto,news_text_content
0,news90370,?Colapsara la industria de la mineria de Bitco...,Salud,Desde Investing.com Espana le invitamos a que ...
1,news82173,Preocupacion por el aumento de los ciberataque...,Salud,La adopcion de herramientas digitales ha impac...
2,news48243,"Logra UTCH 3 de oro,1 de plata y 2 de bronce e...",Sostenibilidad,En el estado de Coahuila se desarrollo el even...
3,news53224,"Emiliano Martinez: ""Los colombianos nos habian...",Sostenibilidad,Emiliano Martinez siempre es tema cuando habla...
4,news59128,El cambio climatico y el exterminio de la huma...,Innovación,La especulacion sobre el exterminio de la huma...
5,news98556,"El Rin se esta reduciendo, poniendo en peligro...",Innovación,Dortmund/Londres (CNN Business) -- El rio Rin ...
6,news96438,Presentaron el Black Friday y Posadas ya vive ...,Regulaciones,La apertura se realizo en Casa de Gobierno y p...
7,news43424,?Por que quedo Boyaca con seis senadores para ...,Regulaciones,"Miguel Angel Barreto Castillo, de Sutatenza; S..."
8,news37975,!A reciclar! Aparte de contribuir con el plane...,Deportes,El crecimiento de una ciudad deja huellas posi...
9,news61002,Bavaria cuenta con 320 familias productoras de...,Deportes,"Usando el modelo de agricultura por contrato, ..."


In [9]:
# Ejemplo cliente sector Industrial
h = recomendador(800000946, 10, df_bd_stemming_f2)
h

,news_id,news_title,Tema_texto,news_text_content
0,news27182,Objetivos generales: ?por que son tan importan...,Macroeconómicas,Definir y tratar de cumplir unos objetivos gen...
1,news84150,Acuerdan alianza estrategica para luchar contr...,Macroeconómicas,La Aduana Nacional y la Federacion de Empresar...
2,news95699,"Alberto Fernandez se reunio con Gustavo Petro,...",Política,"Alberto Fernandez invito a Gustavo Petro , pre..."
3,news13938,"Joe Biden, presidente de EE. UU., da positivo ...",Política,"El presidente de Estados Unidos, Joe Biden, di..."
4,news35467,Palmeiras logro la epica ante Mineiro y sigue ...,Sostenibilidad,El bicampeon Palmeiras clasifico a las semifin...
5,news51867,El Betis planta las bases de su Ciudad Deporti...,Sostenibilidad,Joaquin es el hombre para todo en el Betis . S...
6,news78439,Confirman la muerte de un trabajador por golpe...,Innovación,Un hombre de 58 anos y nacionalidad rumana fal...
7,news51411,Como la CIA rastreo al lider de Al Qaeda duran...,Innovación,Oficiales de inteligencia hicieron un descubri...
8,news64532,Destaca Marina del Pilar distincion del Valle ...,Regulaciones,Ensenada BC.- La region de Valle de Guadalupe ...
9,news14811,Vivian Puerta fracaso en Buen Comienzo y la pr...,Regulaciones,Puerta se formo en Bogota y venia de ser direc...


El recomendador asigna de manera adecuada la priorización de noticias de acuerdo al sector.

### ***4.2 Recomendador: LDA - Lematización***

In [10]:
df_bd_lemat_f2=pd.read_csv('../Data/DC_lematizado_tema_asignado.csv')

Se crea la columna con el sector y se seleccionan las variables más relevantes para el recomendador:

In [11]:
condicion = lambda x: 'Salud' if x in ['ips','comercio de farmaceuticos', 'eps salud prepagada salud','manufactura de productos farmaceuticos'] else 'Industrial'
df_bd_lemat_f2['sector'] = df_bd_lemat_f2['subsec'].apply(condicion)

# Se seleccionana las columnas relevantes
df_bd_lemat_f2 = df_bd_lemat_f2[['nit','sector', 'subsec','news_id', 'news_title', 'news_text_content', 'Tema_texto']]
df_bd_lemat_f2.head(3)

,nit,sector,subsec,news_id,news_title,news_text_content,Tema_texto
0,800000946,Industrial,manufactura de productos de tocador articulos ...,news15986,Guia de Arte y Cultura: semana del 12 al 19 de...,"ARTE Agustina Galindez Quesada exhibe ""Catarsi...",Cultura
1,800000946,Industrial,manufactura de productos de tocador articulos ...,news94698,Bebidas organicas y veganas: novedades de Expo...,Tras dos anos de realizarse en formato virtual...,Cultura
2,800011161,Industrial,automotores,news17955,"No solo habra desfiles el 20 de julio, estos s...","El Dia de la Independencia, este miercoles 20 ...",Cultura


Se revisa cuantas noticias existe por sector:

In [12]:
df_bd_lemat_f2['sector'].value_counts()

sector
Industrial    21697
Salud          1303
Name: count, dtype: int64

Como se dijo durante el stemming, gran parte de la población pertenece a otros sectores diferentes al sector salud.

Ahora, se procede a consumir las funciones creadas cuando se creó el recomendador por stemming:

In [13]:
# Ejemplo cliente sector salud
p = recomendador(860003216, 10, df_bd_lemat_f2)
p

,news_id,news_title,Tema_texto,news_text_content
0,news75416,"Los jovenes talentos de la quimica, reconocido...",Salud,Los galardonados en la XIV Edicion han sido: P...
1,news13052,Investigacion desmiente atribuciones de la vit...,Salud,"DIARIO DEL HUILA, SALUD La revista The New Eng..."
2,news89626,Deforestacion: mas alla de tumbar y sembrar ar...,Sostenibilidad,"Martes, Octubre 4 de 2022 Credito: FCDS La sob..."
3,news59461,El manejo holistico en la ganaderianeuquina,Sostenibilidad,A mediados de este ano inicio la ejecucion de ...
4,news80832,'Paz total': el Clan del Golfo y el posible ac...,Innovación,Escuche aqui nuestro programa 'El primer cafe'...
5,news55311,FOTOS: IKEA abre su primera tienda en Sudameri...,Innovación,Tras cuatro anos de trabajo la primera tienda ...
6,news47050,"Hidrogeno verde, retos y oportunidades en el...",Regulaciones,"+21,000 proyectos en Latinoamerica. +28,600 e..."
7,news55904,"""Reputacion"": en campana politica todo vale | ...",Regulaciones,Reputacion es una serie de seis capitulos que ...
8,news92557,Liga 1 2022: Asi se jugara la fecha 7 del Torn...,Deportes,Programacion de la fecha 7 del Torneo Clausura...
9,news25895,El colombiano Eduard Atuesta marco su primer g...,Deportes,Tras 38 juegos encima y un total de 1.284' min...


In [14]:
# Ejemplo cliente sector Industrial
h = recomendador(800000946, 10, df_bd_lemat_f2)
h

,news_id,news_title,Tema_texto,news_text_content
0,news96556,Peso cubano y mipymes en la mirilla,Macroeconómicas,La proxima apertura de un mercado cambiario pa...
1,news42292,Dilemas y desafios de la economia dominicana e...,Macroeconómicas,La economia dominicana con el covid-19 vio int...
2,news92036,Correos y Grupo ONCE crean un convenio para de...,Política,Correos ha suscrito hoy un convenio marco de c...
3,news40343,"Segun Roy Barreras ""este sera el Congreso de l...",Política,"El senador Roy Barreras, presidente electo de ..."
4,news18686,El cultivo de papa con una fuerte transformacion,Sostenibilidad,Una investigacion de la Maestria en Medio Amb...
5,news63349,Transicion energetica: el futuro esta en manos...,Sostenibilidad,A los militantes ambientalistas les encanta ac...
6,news14602,Estas son las mejores 'startup' colombianas en...,Innovación,Cuando se piensa en conceptos como inteligenci...
7,news91930,Legislador estadounidense pide a Apple y Googl...,Innovación,"El senador Sherrod Brown, presidente del Comit..."
8,news18357,INV: Las empresas pueden hacer inversiones ded...,Regulaciones,INV Agente de Seguros SA de CV es el primer br...
9,news41974,Chilena Colbun considera claves PPA para const...,Regulaciones,"+21,000 proyectos en Latinoamerica. +28,600 e..."


El recomendador asigna de manera adecuada la priorización de noticias de acuerdo al sector.

## ***5. Resultados***

En este apartado se visualizan dos resultados de las sugerencias de las noticias para los clientes, uno del sector salud y otro del sector industrial. La idea es mostrar 10 noticias para cada uno de los modelos y revisar la asertividad de las noticias.

**Sector salud - LDA (Stemming)**

In [15]:
p = recomendador(860003216, 10, df_bd_stemming_f2)
p

,news_id,news_title,Tema_texto,news_text_content
0,news43795,Colombia Productiva lanzo plataforma especiali...,Salud,"Colombia Productiva , entidad adscrita al Mini..."
1,news36936,Binance lanza su primera tarjeta cripto en Lat...,Salud,"Buenos Aires -- Binance, el mayor exchange de ..."
2,news65227,"Cartagena: ""Fue un golpe duro lo sucedido en D...",Sostenibilidad,"Wilder Cartagena, volante nacional, converso c..."
3,news96188,Argentina y Paraguay jugaran por el tercer pue...,Sostenibilidad,MEDIAKIT - MINUTOUNO Director: Jorge Garcia Pr...
4,news55468,Nuevas cuarentenas en China amenazan con debil...,Innovación,Si hay algun indicio de que lo que sucedera a ...
5,news41471,Una alerta desde Canarias previno a la Interpo...,Innovación,"""Necesito saber como las compro, si fue direct..."
6,news34842,Nuevas exposiciones de arte en la ciudad invit...,Regulaciones,"A media cuadra, en el Art Hotel, se encuentra ..."
7,news39595,"Puerto Tirol, una localidad pujante que celebr...",Regulaciones,Hoy celebra los 134 anos Puerto Tirol. Un pueb...
8,news72265,Son 5000 los articulos y 135 las empresas que ...,Deportes,El municipio comunico este miercoles que ya so...
9,news87044,VIU y Fundacion ONCE impulsan la educacion inc...,Deportes,La Universidad Internacional de Valencia y Fun...


**Sector salud - LDA (Lematización)**

In [16]:
p = recomendador(860003216, 10, df_bd_lemat_f2)
p

,news_id,news_title,Tema_texto,news_text_content
0,news92469,Salud niega muerte por falla electrica en el H...,Salud,Un hombre alego este lunes que perdio a su hij...
1,news12835,Laboratorio clinico de la Unipamplona prestara...,Salud,Con distintivo de habilitacion la dependencia ...
2,news61649,CEMEX realiza acciones para reducir emisiones ...,Sostenibilidad,"MONTERREY.- CEMEX, S.A.B. de C.V. (""CEMEX"") in..."
3,news78101,"Los desechos alimenticios humanos son ""una ame...",Sostenibilidad,Los osos polares estan gravemente amenazados p...
4,news66526,Los 30 mejores pijama mujer largo cuidadosamen...,Innovación,Todos somos conscientes de la dificultad a la ...
5,news29721,Liquido de la Encapsulacion de Mercado de los ...,Innovación,Descripcion del reporte Este informe es un est...
6,news33716,El trabajo de una nina wayuu. - Tuu Putchika,Regulaciones,"Esta es la historia de Natalia, una nina que v..."
7,news62178,"Orden DEF/711/2022, de 18 de julio, por la que...",Regulaciones,Primera. Objeto. Estas normas tienen por objet...
8,news44714,Ministra del Deporte enfrento a la Dimayor y m...,Deportes,Uno de los deportes que ha generado polemica d...
9,news89026,GOLPERU transmitio: Alianza Lima 1-0 Alianza A...,Deportes,/ Noticias Informacion basada en hechos y veri...


**Sector Industrial - LDA (Stemming)**

In [17]:
h = recomendador(800000946, 10, df_bd_stemming_f2)
h

,news_id,news_title,Tema_texto,news_text_content
0,news61284,Pedro Castillo se compromete con el empresaria...,Macroeconómicas,"Pretende a corto plazo, concretar una alianza ..."
1,news88809,?Cual es la salida para la crisis en Argentina...,Macroeconómicas,(CNN Espanol) -- Argentina busca una salida pa...
2,news94691,?Ley seca para la posesion de Petro? Esta fue ...,Política,El Gobierno encabezado por el presidente Ivan ...
3,news72883,Proponen que Informe de Comision de la Verdad ...,Política,"Colombia En entrevista con La W, el senador de..."
4,news17681,[Video] No te pierdas el gol con el que la sel...,Sostenibilidad,La seleccion Colombia femenina se encuentra en...
5,news18819,Palmeiras recupera el liderato y desbanca a un...,Sostenibilidad,"Sao Paulo, 18 jul (EFE).- Palmeiras derroto es..."
6,news85909,Ucrania: millones de toneladas de alimentos si...,Innovación,La guerra en Ucrania esta proxima a cumplir se...
7,news85033,Lamenta Cuba perdidas humanas y danos por lluv...,Innovación,"La Habana, 2 ago.- El canciller de Cuba, Brun..."
8,news29534,SuperSociedades reanudara este jueves la audie...,Regulaciones,La Superintendencia de Sociedades definira est...
9,news12006,?Por que la empresa de agua Eicviro retoma el ...,Regulaciones,La Superintendencia de Servicios Publicos Domi...


**Sector Industrial - LDA (Lematización)**

In [18]:
h = recomendador(800000946, 10, df_bd_lemat_f2)
h

,news_id,news_title,Tema_texto,news_text_content
0,news26959,"A junio, ahorro pensional de los colombianos h...",Macroeconómicas,En medio de un panorama en el que las economia...
1,news59059,Inversionistas perdieron fe en la capacidad de...,Macroeconómicas,El Peru es el segundo mayor productor mundial ...
2,news68636,Condenable el uso de obras en campana electora...,Política,"Por un lado, el presidente de la Republica, Ma..."
3,news70250,Demandan en Panama poner mayor enfasis en medi...,Política,Segun un comunicado de la Coalicion pro-transp...
4,news32591,Confirman otro hallazgo de gas costa afuera en...,Sostenibilidad,El gas es uno de los energeticos fundamentales...
5,news74813,Sector agricola requiere de tecnologia y finan...,Sostenibilidad,Caracas. El cambio climatico amenaza con volve...
6,news61798,?Que tanto potencial tiene el modelo de carro ...,Innovación,Aunque en Colombia todavia es un servicio inci...
7,news98033,Grupo Hi-Tec celebra su Demo Day A&G Plastic M...,Innovación,Grupo Hi-Tec celebro el Demo Day de su divisio...
8,news17292,Ciudad de Mexico construira segunda planta d...,Regulaciones,"+21,000 proyectos en Latinoamerica. +28,600 e..."
9,news13025,El enfoque del nuevo director Seccional de Fis...,Regulaciones,"DIARIO DEL HUILA, PRIMER PLANO Por: Johan Edua..."
